In [1]:
import os
import gc
import re
import json
import pickle
import datetime
from tqdm import tqdm

import numpy as np
import pandas as pd
pd.options.display.max_columns = None

from typing import Union

import seaborn
import matplotlib.pyplot as plt
plt.style.use('seaborn-darkgrid')
%config InlineBackend.figure_formats = {'png', 'retina'}

from scipy.stats import linregress

from sklearn import preprocessing
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error

import lightgbm as lgb

In [5]:
VERSION = 'v00000'
SEED = 42

In [6]:
train = pd.read_pickle('/Users/rui/Documents/repositories/m5_forecasting_accuracy/v01000/features/drop_null_rows.pkl')

In [7]:
print(train.shape)
train.head()

(35605264, 63)


,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,month,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,quarter,weekofyear,day,dayofweek,is_quarter_end,is_quarter_start,is_month_end,is_month_start,is_weekend,sell_price,sell_price_rate_by_wm_yr_wk__item_id,sell_price_rate_by_wm_yr_wk__area__item_id,sell_price_momentum,sales_lag_t28,sales_lag_t29,sales_lag_t30,sales_lag_t32,sales_lag_t33,sales_lag_t34,sales_rolling_STD_t7,sales_rolling_STD_t30,sales_rolling_STD_t60,sales_rolling_STD_t90,sales_rolling_STD_t180,sales_rolling_MEAN_t7,sales_rolling_MEAN_t30,sales_rolling_MEAN_t60,sales_rolling_MEAN_t90,sales_rolling_MEAN_t180,sales_rolling_MIN_t7,sales_rolling_MIN_t30,sales_rolling_MIN_t60,sales_rolling_MAX_t7,sales_rolling_MAX_t30,sales_rolling_MAX_t60,sales_rolling_ZeroRatio_t7,sales_rolling_ZeroCount_t7,sales_rolling_ZeroRatio_t14,sales_rolling_ZeroCount_t14,sales_rolling_ZeroRatio_t30,sales_rolling_ZeroCount_t30,sales_rolling_ZeroRatio_t60,sales_rolling_ZeroCount_t60,sales_rolling_SKEW_t30,sales_rolling_KURT_t30
6524867,HOBBIES_1_008_CA_1_validation,1444,3,1,0,0,d_215,8,2011-08-31,11131,8,5,3,3,1,0,0,0,3,35,31,2,0,0,1,0,0,0.500000,1.000000,1.0,1.0,37.0,7.0,0.0,3.0,0.0,18.0,13.460938,8.398438,8.554688,8.203125,7.117188,9.859375,10.234375,7.683594,6.535156,4.871094,0.0,0.0,0.0,37.0,37.0,37.0,0.714355,2.0,0.785645,3.0,0.899902,3.0,0.666504,20.0,1.278320,2.142578
6524868,HOBBIES_1_009_CA_1_validation,1445,3,1,0,0,d_215,0,2011-08-31,11131,8,5,3,3,1,0,0,0,3,35,31,2,0,0,1,0,0,1.769531,1.031250,1.0,1.0,0.0,7.0,2.0,0.0,1.0,4.0,2.572266,3.589844,3.037109,2.933594,2.697266,2.572266,2.267578,2.166016,2.099609,2.060547,0.0,0.0,0.0,7.0,18.0,18.0,0.714355,2.0,0.714355,4.0,0.600098,12.0,0.600098,24.0,3.177734,12.648438
6524869,HOBBIES_1_010_CA_1_validation,1446,3,1,0,0,d_215,2,2011-08-31,11131,8,5,3,3,1,0,0,0,3,35,31,2,0,0,1,0,0,2.970703,1.000000,1.0,1.0,4.0,2.0,0.0,0.0,0.0,0.0,1.573242,0.958984,0.860840,0.822754,0.669434,1.142578,0.666504,0.733398,0.555664,0.305664,0.0,0.0,0.0,4.0,4.0,4.0,0.428467,4.0,0.357178,9.0,0.433350,17.0,0.516602,29.0,1.750000,3.664062
6524871,HOBBIES_1_012_CA_1_validation,1448,3,1,0,0,d_215,0,2011-08-31,11131,8,5,3,3,1,0,0,0,3,35,31,2,0,0,1,0,0,6.269531,1.000000,1.0,1.0,2.0,1.0,1.0,0.0,1.0,0.0,0.755859,0.739746,0.743164,0.722168,0.791016,0.714355,0.733398,0.583496,0.533203,0.572266,0.0,0.0,0.0,2.0,2.0,2.0,0.571289,3.0,0.571289,6.0,0.566895,13.0,0.433350,34.0,0.480469,-0.972168
6524874,HOBBIES_1_015_CA_1_validation,1451,3,1,0,0,d_215,0,2011-08-31,11131,8,5,3,3,1,0,0,0,3,35,31,2,0,0,1,0,0,0.700195,1.005859,1.0,1.0,2.0,5.0,7.0,5.0,1.0,12.0,4.386719,7.816406,7.214844,7.355469,6.750000,6.285156,7.398438,7.816406,7.378906,6.187500,1.0,0.0,0.0,12.0,40.0,40.0,1.000000,0.0,0.928711,1.0,0.899902,3.0,0.933105,4.0,2.718750,9.992188


In [9]:
target = 'sales'
cols_to_drop = ['id', 'wm_yr_wk', 'd', 'date'] + [target]
feature = train.columns.tolist()
feature = [f for f in feature if f not in cols_to_drop]

In [13]:
class RondomSeed_LGBM_Model():
    def __init__(self, data, feature, target, n_fold, test_days, max_train_days, model_param, train_param, weight_col=None):
        self.n_fold = n_fold
        train_dataset, valid_dataset = self.split_data(data, feature, target, test_days, max_train_days, weight_col)
        self.models = self.fit(train_dataset, valid_dataset, model_param, train_param)
        
    
    def split_data(self, data, feature, target, test_days, max_train_days, weight_col):
        latest_date = data['date'].max()
        oldest_valid_date = latest_date - datetime.timedelta(days=test_days)
        valid_mask = (data["date"] > oldest_valid_date)
        oldest_train_date = oldest_valid_date - datetime.timedelta(days=max_train_days)
        train_mask = (data["date"] > oldest_train_date) & (data["date"] <= oldest_valid_date)
        
        train_X, train_y = data.loc[train_mask, feature], data.loc[train_mask, target]
        valid_X, valid_y = data.loc[valid_mask, feature], data.loc[valid_mask, target]
        
        train_dataset = lgb.Dataset(train_X, label=train_y)
        valid_dataset = lgb.Dataset(valid_X, label=valid_y, reference=train_dataset)
        
        if weight_col:
            train_dataset.set_weight(data.loc[train_mask, weight_col])
            valid_dataset.set_weight(data.loc[valid_mask, weight_col])
        
        print('Train DataFrame Size:', train_mask.sum())
        print('Valid DataFrame Size:', valid_mask.sum())
        return train_dataset, valid_dataset
          

    def fit(self, train_dataset, valid_dataset, model_param, train_param):
        models = []
        for n in range(self.n_fold):
            print(f"\n{n + 1} of {self.n_fold} Fold:\n")
            model_param['seed'] = model_param['seed'] + 1
            model = lgb.train(
                model_param,
                train_dataset,
                valid_sets=[train_dataset, valid_dataset],
                valid_names=["train", "valid"],
                **train_param
            )
            models.append(model)
        return models

    def get_models(self):
        return self.models

    def predict(self, data):
        models = self.get_models()
        return [m.predict(data.values, num_iteration=m.best_iteration) for m in models]

    def save_importance(self, filepath, max_num_features=50, figsize=(20, 25), plot=False):
        models = self.get_models()
        # Define Feature Importance DataFrame.
        imp_df = pd.DataFrame(
            [m.feature_importance() for m in models],
            columns=models[0].feature_name()
        ).T
        imp_df['AVG_Importance'] = imp_df.iloc[:, :len(models)].mean(axis=1)
        imp_df['STD_Importance'] = imp_df.iloc[:, :len(models)].std(axis=1)
        imp_df.sort_values(by='AVG_Importance', inplace=True)
        # Plot Importance DataFrame.
        plt.figure(figsize=figsize)
        imp_df[-max_num_features:].plot(
            kind='barh', title='Feature importance', figsize=figsize,
            y='AVG_Importance', xerr='STD_Importance', align="center"
        )
        if plot:
            plt.show()
        plt.savefig(filepath)
        plt.close('all')

In [14]:
def estimate_rmsle(actual, preds):
    return np.sqrt(mean_squared_log_error(actual, preds))

def lgbm_rmsle(preds, data):
    return 'RMSLE', estimate_rmsle(preds, data.get_label()), False

In [ ]:
# define custom loss function
def custom_asymmetric_train(y_pred, y_true):
    y_true = y_true.get_label()
    residual = (y_true - y_pred).astype("float")
    grad = np.where(residual < 0, -2 * residual, -2 * residual * 1.15)
    hess = np.where(residual < 0, 2, 2 * 1.15)
    return grad, hess

# define custom evaluation metric
def custom_asymmetric_valid(y_pred, y_true):
    y_true = y_true.get_label()
    residual = (y_true - y_pred).astype("float")
    loss = np.where(residual < 0, (residual ** 2) , (residual ** 2) * 1.15) 
    return "custom_asymmetric_eval", np.mean(loss), False

In [ ]:
n_fold = 1
max_train_days = 2 * 365
test_days = 180

model_param = {
    "boosting_type": "gbdt",
    "metric": "None",
    "objective": "poisson",
    "seed": SEED,
    "force_row_wise" : True,
    "learning_rate": 0.3,
    'num_leaves': 80,
    'min_data_in_leaf': 50,
    "bagging_fraction": 0.8,
    "bagging_freq": 10,
    "feature_fraction": 0.8,
    "verbosity": -1
}

train_param = {
    "num_boost_round": 1000,
    "early_stopping_rounds": 50,
    "verbose_eval": 100,
    "feval": custom_asymmetric_valid
}

lgbm_model = RondomSeed_LGBM_Model(train, feature, target, n_fold, test_days, max_train_days, model_param, train_param)

Train DataFrame Size: 17565234
Valid DataFrame Size: 4976503

1 of 1 Fold:

Training until validation scores don't improve for 50 rounds
[100]	train's RMSLE: 0.537364	valid's RMSLE: 0.595318
[200]	train's RMSLE: 0.532692	valid's RMSLE: 0.593887
[300]	train's RMSLE: 0.529668	valid's RMSLE: 0.592944
